# Sentiment Classification of Hotel Reviews

This notebook loads customer hotel reviews, classifies their sentiment using `analyze_sentiment` (GPT-4o),
and compares the predicted sentiment with the original `survey_sentiment` label.

In [5]:
import json
import sys
import os
from pathlib import Path

# Add scripts/ to path so we can import the recommender module
NOTEBOOK_DIR = Path(os.path.abspath("")).resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent
sys.path.insert(0, str(PROJECT_ROOT / "scripts"))

print(f"Project root: {PROJECT_ROOT}")

Project root: C:\git\wwsi-2026-genai


In [6]:
import importlib
recommender = importlib.import_module("ner-trip-recommender")

c:\git\wwsi-2026-genai\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading NER model...
NER model loaded.
Initializing ChromaDB vector store...
Indexed 154 trips.


## Load data

- `customer_surveys_hotels_1k.json` — contains `id`, `review`, `customer_satisfaction_score`, `survey_sentiment`
- `customer_surveys_hotels_1k_ner.json` — contains `id`, `text` (same review), `entities`

We join both on `id` to get the full picture.

In [7]:
DATA_DIR = PROJECT_ROOT / "data"

with open(DATA_DIR / "customer_surveys_hotels_1k.json", "r", encoding="utf-8") as f:
    surveys = json.load(f)

with open(DATA_DIR / "customer_surveys_hotels_1k_ner.json", "r", encoding="utf-8") as f:
    surveys_ner = json.load(f)

# Build lookup by id
survey_by_id = {s["id"]: s for s in surveys}

print(f"Loaded {len(surveys)} surveys and {len(surveys_ner)} NER records")
print(f"Sample keys (surveys): {list(surveys[0].keys())}")
print(f"Sample keys (NER):     {list(surveys_ner[0].keys())}")

Loaded 1000 surveys and 1000 NER records
Sample keys (surveys): ['id', 'review', 'customer_satisfaction_score', 'survey_sentiment']
Sample keys (NER):     ['id', 'text', 'entities']


## Classify sentiment

For each review we call `analyze_sentiment` which returns:
```json
{"positive_sentiment": true/false, "reasoning": "..."}
```

We map `positive_sentiment` → `"positive"` / `"negative"` and store it as `predicted_sentiment`.

In [5]:
results = []

for i, record in enumerate(surveys_ner):
    review_id = record["id"]
    review_text = record["text"]
    survey = survey_by_id.get(review_id, {})

    sentiment = recommender.analyze_sentiment(review_text)
    predicted = "positive" if sentiment["positive_sentiment"] else "negative"

    results.append({
        "id": review_id,
        "review": review_text,
        "customer_satisfaction_score": survey.get("customer_satisfaction_score"),
        "survey_sentiment": survey.get("survey_sentiment"),
        "predicted_sentiment": predicted,
    })

    if (i + 1) % 50 == 0 or i == 0:
        print(f"[{i+1}/{len(surveys_ner)}] id={review_id[:8]}... survey={survey.get('survey_sentiment')} predicted={predicted}")

print(f"\nDone. Classified {len(results)} reviews.")

[1/1000] id=7a823fbc... survey=neutral predicted=negative
[50/1000] id=95a76512... survey=neutral predicted=negative
[100/1000] id=006f2b88... survey=positive predicted=positive
[150/1000] id=130b35fa... survey=negative predicted=negative
[200/1000] id=1a3362fe... survey=negative predicted=negative
[250/1000] id=565ce5ec... survey=negative predicted=negative
[300/1000] id=a006838a... survey=positive predicted=positive
[350/1000] id=fc382184... survey=negative predicted=negative
[400/1000] id=bf35ed23... survey=positive predicted=positive
[450/1000] id=3457128d... survey=negative predicted=negative
[500/1000] id=15520646... survey=positive predicted=positive
[550/1000] id=8e73d636... survey=negative predicted=positive
[600/1000] id=9e5d6166... survey=positive predicted=positive
[650/1000] id=86fe69d7... survey=neutral predicted=positive
[700/1000] id=52c0130f... survey=positive predicted=positive
[750/1000] id=2260639c... survey=negative predicted=negative
[800/1000] id=bd3d1a04... surv

## Save results

In [6]:
OUTPUT_PATH = DATA_DIR / "sentiment_classification_results.json"

with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"Saved {len(results)} records to {OUTPUT_PATH}")

Saved 1000 records to C:\git\wwsi-2026-genai\data\sentiment_classification_results.json


In [8]:
# Load results from file (use this cell to skip the classification step above)
with open(DATA_DIR / "sentiment_classification_results.json", "r", encoding="utf-8") as f:
    results = json.load(f)

print(f"Loaded {len(results)} results from sentiment_classification_results.json")

Loaded 1000 results from sentiment_classification_results.json


## Quick stats

In [7]:
from collections import Counter

survey_counts = Counter(r["survey_sentiment"] for r in results)
predicted_counts = Counter(r["predicted_sentiment"] for r in results)

match = sum(1 for r in results if r["survey_sentiment"] == r["predicted_sentiment"])
total = len(results)

print("Survey sentiment distribution:")
for label, count in survey_counts.most_common():
    print(f"  {label}: {count}")

print(f"\nPredicted sentiment distribution:")
for label, count in predicted_counts.most_common():
    print(f"  {label}: {count}")

print(f"\nExact match (survey == predicted): {match}/{total} ({match/total*100:.1f}%)")
print(f"Note: survey_sentiment has 3 classes (positive/negative/neutral),")
print(f"      predicted_sentiment has 2 classes (positive/negative).")

Survey sentiment distribution:
  positive: 400
  negative: 400
  neutral: 200

Predicted sentiment distribution:
  negative: 550
  positive: 450

Exact match (survey == predicted): 772/1000 (77.2%)
Note: survey_sentiment has 3 classes (positive/negative/neutral),
      predicted_sentiment has 2 classes (positive/negative).


## Evaluation (neutral excluded)

`survey_sentiment` has 3 classes (positive/negative/neutral) but our model predicts only 2 (positive/negative).
We exclude the 200 neutral reviews for a fair binary evaluation on 800 samples.

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Filter out neutral reviews
binary_results = [r for r in results if r["survey_sentiment"] != "neutral"]
y_true = [r["survey_sentiment"] for r in binary_results]
y_pred = [r["predicted_sentiment"] for r in binary_results]

print(f"Samples: {len(y_true)} (excluded {len(results) - len(y_true)} neutral)\n")

accuracy  = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, pos_label="positive")
recall    = recall_score(y_true, y_pred, pos_label="positive")
f1        = f1_score(y_true, y_pred, pos_label="positive")

print(f"Accuracy:  {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1-score:  {f1:.3f}")

print(f"\nFull classification report:")
print(classification_report(y_true, y_pred, digits=3))

Samples: 800 (excluded 200 neutral)

Accuracy:  0.965
Precision: 0.989
Recall:    0.940
F1-score:  0.964

Full classification report:
              precision    recall  f1-score   support

    negative      0.943     0.990     0.966       400
    positive      0.989     0.940     0.964       400

    accuracy                          0.965       800
   macro avg      0.966     0.965     0.965       800
weighted avg      0.966     0.965     0.965       800



## Confusion Matrix (binary)

In [10]:
import plotly.figure_factory as ff

labels = ["positive", "negative"]
cm = confusion_matrix(y_true, y_pred, labels=labels)
cm_text = [[str(val) for val in row] for row in cm]

fig_cm = ff.create_annotated_heatmap(
    z=cm,
    x=[f"pred: {l}" for l in labels],
    y=[f"true: {l}" for l in labels],
    annotation_text=cm_text,
    colorscale="Blues",
    showscale=True,
)
fig_cm.update_layout(
    title="Confusion Matrix (binary — neutral excluded)",
    xaxis_title="Predicted",
    yaxis_title="Actual",
    height=400,
)
fig_cm.show()